<table style="width: 100%">
    <tr style="background: #ffffff">
        <td style="padding-top:25px;width: 180px"><img src="https://mci.edu/templates/mci/images/logo.svg" alt="Logo"></td>
        <td style="width: 100%">
            <div style="text-align:right; width: 100%; text-align:right"><font style="font-size:38px"><b>Data Science</b></font></div>
            <div style="padding-top:0px; width: 100%; text-align:right"><font size="4"><b></b></font></div>
        </td>
    </tr>
</table>

---

# 9 Image Classification with Deep Learning

This notebook is a very short introduction to image classification using pre-trained neural networks and `fastai`. The notebook builds on the [tutorial from Jeremy Howard](https://www.kaggle.com/code/jhoward/is-it-a-bird-creating-a-model-from-your-own-data/notebook). You can read the [post by 
Hafiz Ahmad Hassan](https://medium.com/@l154359/deep-learning-module-ii-fast-ai-series-image-classification-1-d21be0198aa7) if You are interested in more details.

## Used Packages

We will use fast.ai. They have the motto of *"Making neural nets uncool again"*. To-do so they developed a framework, which makes it very easy to use state of the art deep-learning models. The concept is similar to `seaborn` which abstracts from `matplotlib` to make nicer looking plots more conveniently.

We start by installing the following Python packages:

In [1]:
# A package to use the duck duck go search engine from within python
!pip install duckduckgo_search

# The fast.ai library
!pip install fastai
!pip install fastcore

# A library to download pictures easily
!pip install fastdownload


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: C:\Users\JLHUBER\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: C:\Users\JLHUBER\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: C:\Users\JLHUBER\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: C:\Users\JLHUBER\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


## Creating the Data-Set

First we need some images to fine-tune our model. Note that we will use a pre-trained model as the training ot the thousands of weights (parameters) would take to much data an time before we would get good results.



## Downloading the Data

We will automatically download the pictures matching a search term from the duckduckgo-search engine:

In [2]:
from duckduckgo_search import ddg_images
from fastcore.all import *

def search_images(term, max_images=30):
    """A function that download the first n pictures of a search term and returns a list of the picture urls"""
    print(f"Searching for '{term}'")

    return L(ddg_images(term, max_results=max_images)).itemgot('image')

In [4]:
# Testing the function
#NB: `search_images` depends on duckduckgo.com, which doesn't always return correct responses.
#    If you' get a JSON error, just try running it again (it may take a couple of tries).
urls = search_images('bird photos', max_images=1)
urls[0]

Searching for 'bird photos'


RateLimitException: _get_url() https://duckduckgo.com

Next, we download the image in the `images`-subfolder auf our directory:

In [ ]:
from fastdownload import download_url
dest = 'data/images/bird.jpg'
download_url(urls[0], dest, show_progress=False)

If we want to check the image within the notebook, we can print a thumbnail:

You can imagine that the predicted variable $y$ is the label of the image. The label is the name of the folder, where the image is stored. In particular, the label must be dummy encoded for the model to work.

The predictor $X$ is the image itself, which the color values of each pixel.

In [ ]:
from fastai.vision.all import *
im = Image.open(dest)
im.to_thumb(256,256)

The same works for other search terms. In this example, we want to build a classifier that can spot birds.

In [ ]:
download_url(search_images('forest photos', max_images=1)[0], 'data/images/forest.jpg', show_progress=False)
Image.open('data/images/forest.jpg').to_thumb(256,256)

### Downloading Data with different Characteristics and Labels

We want to make sure, that the training data is divers and we do not use the same type of bird over and over again. For this reason, we do not only download the first pictures of birds which could be comics of angry birds instead of ral photos (which are probably bright with sky in the background) and forests (probably dark), but also include different lightning conditions in the search term.

Hence, we adapt our search term `'<bird/forest> photo'` and download multiple pictures with different search terms.

In this case, we have a very easy approach to label the data. We just label everything that has `bird` in the search term as a bird and everything with `forest` as a forest. We also do not need to have a list of the labels, as, we just store all birds in the `bird_or_not/bird` subfolder and vice versa.

In [ ]:
# Define the classes
searches = 'forest','bird'

# Define where to store the data
path = Path('data/images/bird_or_not')

# Sleep time so that the search engine does not block us, because of too many requests
from time import sleep

# For all class labels
for class_label in searches:
    # select the folder to store them
    dest = (path/class_label)
    # create the folder if it does not exists
    dest.mkdir(exist_ok=True, parents=True)
    # Use the download function to store the picture
    download_images(dest, urls=search_images(f'{class_label} photo'))
    sleep(10)  # Pause between searches to avoid over-loading server
    download_images(dest, urls=search_images(f'{class_label} sun photo'))
    sleep(10)
    download_images(dest, urls=search_images(f'{class_label} shade photo'))
    sleep(10)
    resize_images(path/class_label, max_size=400, dest=path/class_label)

Now, we do a quick check to remove corrupted files and remove them. 

In [ ]:
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
len(failed)

### Create a Data Block for the Model to train with

Instead of two Dataframes (`X` for the features and `y` for the predicted variable) fast.ai uses a [DataBlock-Objekt](https://docs.fast.ai/data.block.html#datablock), to describe all the data to be loaded for training.


In [ ]:
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock),         # Predictor (items) are images, predicted variable are categories
    get_items=get_image_files,                  # The images must be loaded as image
    splitter=RandomSplitter(valid_pct=0.2, seed=42),    # Use 20% of randomly selected pictures for validation
    get_y=parent_label,                         # To get the y-labels use the name of the parent directory
    item_tfms=[Resize(192, method='squish')]    # Transform the data by resizing them to 192x192 pixels and by squishing them
).dataloaders(path, bs=32)                      # Load the date from the path = 'bird_or_not' in a batch size of 32



We can show the data in a table:

In [ ]:
dls.show_batch(max_n=6)                         # Show the first 6 items

Depending in when You downloaded Your pictures You can see, what training data we created. From the squishing some bird might be a litte fat or too lean. However, the ANN will still be able to work with that. We will also have some faulty pictures like pictures of lamps with birds in this example. Hence, in some cases manual sorting might improve the results. 

Also note, that we resize the image a lot. This will work fine in this use case, but not in use cases with more focus an detail.

![](https://i.imgur.com/E6ZuClP.png)


## Training the Model

For training we can select a pre-configured [vision learner](https://docs.fast.ai/vision.learner.html#vision_learner) to which we pass our training-and-validation data `dls`. These models have been trained on thousands of images and are able to classify images with a high accuracy. However, we need to fine-tune the model to our specific use case.

We also select an error metric like the accuracy and a pre-trained network. [ResNet18](https://arxiv.org/pdf/1512.03385.pdf) is a commonly used model that was pre-trained on thousands of pictures and is rather small compared to other models. Hence, it is a good starting point for our use case.

![](https://www.researchgate.net/profile/Muhammed-Enes-Atik-2/publication/349241995/figure/fig2/AS:991139192643586@1613317406497/ResNet-18-architecture-20-The-numbers-added-to-the-end-of-ResNet-represent-the_W640.jpg)

In [ ]:
# Select pre-trained model and download it
learn = vision_learner(dls, resnet18, metrics=accuracy)

Next we train the model over a given number of runs through the entire training data set (epochs) until a stopping criteria is reached. 

In [ ]:
# Fine-tune the model
learn.fine_tune(3)

## Testing the Model

Finally we can test the model with the images we downloaded first.

In [ ]:
# Make a prediction
predicted_class,_,probs = learn.predict(PILImage.create('images/bird.jpg'))
print(f"This is a: {predicted_class}.")
print(f"Probability it's a bird: {probs[0]:.4f}") 

In [ ]:
# Make a prediction
predicted_class,_,probs = learn.predict(PILImage.create('images/forest.jpg'))
print(f"This is a: {predicted_class}.")
print(f"Probability it's a bird: {probs[0]:.4f}")

In [ ]:
download_url(search_images('Donald Trump', max_images=1)[0], 'images/trump.jpg', show_progress=False)
Image.open('images/trump.jpg').to_thumb(256,256)

In [ ]:
# Make a prediction
predicted_class,_,probs = learn.predict(PILImage.create('images/trump.jpg'))
print(f"This is a: {predicted_class}.")
print(f"Probability it's a bird: {probs[0]:.4f}")
print(f"Probability it's a forest: {probs[1]:.4f}")

✍️ **Task**

- think of another use case for image classification
- copy the notebook and adapt it to Your use case
- e.g., classify images of different pengiun species